In [1]:
import numpy as np
import xarray as xr
from matplotlib import pyplot as plt
import netCDF4
import numpy
import xarray
from netCDF4 import Dataset
import netCDF4 as nc
import wget      
from pathlib import Path

In [2]:
#Saved location
ifile = 'C:/Users/user/Research/Research Code/CNN/Data/'

# Downloaded dataset 1854/01-2022/12

In [3]:
url = "http://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/noaa.ersst.v5/sst.mnmean.nc"
## eliminate a pointless variable that makes some processes more difficult.
ds = xr.open_dataset(url, drop_variables=["time_bnds"], chunks={'time':1})
## will take a couple of seconds to finish
ds = ds.sel(time=slice("1854", "2022")).load()

In [4]:
##Saved file
ds.to_netcdf(ifile+'sst.mnmean.nc')

In [6]:
##Preparing data: drop unused data and add lev dimension 
ds = xr.open_dataset(ifile+'sst.mnmean.nc')
time_bnds = ds.variables['time_bnds'][:].squeeze(dim=None)
x = ds.drop('time_bnds', dim = None)
lev_coords = np.zeros(1)
ds_add_lev = x.expand_dims({"lev": lev_coords})
sst = ds_add_lev.transpose('time','lat','lon','lev')
sst.to_netcdf(ifile+'ersst_add_level_1854_2022')
##Saved adjusted data
sst.to_netcdf(ifile+'nonnormalized_ersst_1854_2022.nc')

# Adjusted Coordinates

In [7]:
data = xr.open_dataarray(ifile+'nonnormalized_ersst_1854_2022.nc', chunks={'time':1})
data = xr.DataArray(data = data, coords={'time':data.time,
                                       'lat':data.lat,
                                       'lon':data.lon,
                                       'lev':data.lev},dims=data.dims)
print(data)

<xarray.DataArray 'sst' (time: 2028, lat: 89, lon: 180, lev: 1)>
dask.array<open_dataset-ed75f9508fd571d1c16b2369219b127asst, shape=(2028, 89, 180, 1), dtype=float32, chunksize=(1, 89, 180, 1), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1854-01-01 1854-02-01 ... 2022-12-01
  * lat      (lat) float32 88.0 86.0 84.0 82.0 80.0 ... -82.0 -84.0 -86.0 -88.0
  * lon      (lon) float32 0.0 2.0 4.0 6.0 8.0 ... 350.0 352.0 354.0 356.0 358.0
  * lev      (lev) float64 0.0
Attributes:
    long_name:     Monthly Means of Sea Surface Temperature
    units:         degC
    var_desc:      Sea Surface Temperature
    level_desc:    Surface
    statistic:     Mean
    dataset:       NOAA Extended Reconstructed SST V5
    parent_stat:   Individual Values
    actual_range:  [-1.8     42.32636]
    valid_range:   [-1.8 45. ]


# Set the study area

In [8]:
data = data.isel(lat = (data.lat<66.0)&(data.lat>0.0))
data = data.isel(lon = (data.lon<250.0)&(data.lon>120.0))

In [9]:
data

<xarray.DataArray 'sst' (time: 2028, lat: 32, lon: 64, lev: 1)>
dask.array<getitem, shape=(2028, 32, 64, 1), dtype=float32, chunksize=(1, 32, 64, 1), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1854-01-01 1854-02-01 ... 2022-12-01
  * lat      (lat) float32 64.0 62.0 60.0 58.0 56.0 ... 10.0 8.0 6.0 4.0 2.0
  * lon      (lon) float32 122.0 124.0 126.0 128.0 ... 242.0 244.0 246.0 248.0
  * lev      (lev) float64 0.0
Attributes:
    long_name:     Monthly Means of Sea Surface Temperature
    units:         degC
    var_desc:      Sea Surface Temperature
    level_desc:    Surface
    statistic:     Mean
    dataset:       NOAA Extended Reconstructed SST V5
    parent_stat:   Individual Values
    actual_range:  [-1.8     42.32636]
    valid_range:   [-1.8 45. ]

In [10]:
##Saved file
data.to_netcdf(ifile+'nonnorm_Datapreprcess_ersst_1854_2022.nc')

# END